## Data Preprocessing

### Project Description:

Using pyspark, due to the large size of the data sets, the following code seeks to join two yelp data sets consisting of businesses, reviews, and ratings for restaurants in Toronto.

The data can be found at https://www.yelp.com/dataset .

In [1]:
# Import packages
import findspark
findspark.init("/Users/joseppbenvenuto/spark-3.0.1-bin-hadoop2.7")
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()
import pandas as pd
import numpy as np

In [16]:
# View schema of data set
reviews = spark.read.json('Data/Yelp_Ontario_Restaurant_Review_Data.json')
reviews_schema = reviews.printSchema()
reviews_schema

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



The above schema describes the data types of each feature found in the data set.

In [17]:
# View data set
reviews.createOrReplaceTempView('reviews')
reviews_sql = spark.sql("""SELECT business_id, stars, text, user_id
                           FROM reviews""")

reviews_sql.show(10)

+--------------------+-----+--------------------+--------------------+
|         business_id|stars|                text|             user_id|
+--------------------+-----+--------------------+--------------------+
|-MhfebM0QIsKt87iD...|  2.0|As someone who ha...|OwjRMXRC0KyPrIlcj...|
|lbrU8StCq3yDfr-QM...|  1.0|I am actually hor...|nIJD_7ZXHq-FX8byP...|
|HQl28KMwrEKHqhFrr...|  5.0|I love Deagan's. ...|V34qejxNsCbcgD8C0...|
|5JxlZaqCnk1MnbgRi...|  1.0|Dismal, lukewarm,...|ofKDkJKXSKZXu5xJN...|
|IS4cv902ykd8wj1TR...|  4.0|Oh happy day, fin...|UgMW8bLE0QMJDCkQ1...|
|nlxHRv1zXGT0c0K51...|  5.0|This is definitel...|5vD2kmE25YBrbayKh...|
|Pthe4qk5xh4n-ef-9...|  5.0|Really good place...|aq_ZxGHiri48TUXJl...|
|FNCJpSn0tL9iqoY3J...|  5.0|Awesome office an...|dsd-KNYKMpx6ma_sR...|
|e_BiI4ej1CW1F0EyV...|  5.0|Most delicious au...|P6apihD4ASf1vpPxH...|
|Ws8V970-mQt2X9CwC...|  4.0|I have been here ...|jOERvhmK6_lo_XGUB...|
+--------------------+-----+--------------------+--------------------+
only s

The above table represents all the businesses under review, reviews, star ratings, and users.

In [18]:
# View schema of data set 
restaurants = spark.read.csv('Data/Yelp_Ontario_Restaurant_Data.csv', header = True)
restaurants_schema = restaurants.printSchema()
restaurants_schema

root
 |-- business_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- attributes: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- hours: string (nullable = true)



The above schema describes the data types of each feature found in the data set.

In [19]:
# View data set
restaurants.createOrReplaceTempView('restaurants')
restaurants_sql = spark.sql("""SELECT business_id, name
                               FROM restaurants""")

restaurants_sql.show(10)

+--------------------+--------------------+
|         business_id|                name|
+--------------------+--------------------+
|0QjROMVW9ACKjhSEf...|    Mi Mi Restaurant|
|37kk0IW6jL7ZlxZF6...|              Edulis|
|Nxg73OigmRQQq0d1p...|   Xe Lua Restaurant|
|K5Q2vkF5UpytV9Q1r...|          Akira Back|
|q0hAKzn_LmyUiScCu...|  Korean Grill House|
|IfUkm42SB9D0m_Zbk...|The Fish Store & ...|
|N8jmyJCk-LjO9o1K9...|  Harlem Underground|
|7ZBh-3wWVQ5zkd6KZ...|       Cibo Wine Bar|
|OURoudNtFXyd7RTtl...|        Café Pamenar|
|aRqxT28Tdz2aDiZ7V...| White Brick Kitchen|
+--------------------+--------------------+
only showing top 10 rows



The above table represents all the businesses and business names under review.

In [20]:
# Use SQL to join the two data sets on business id
full_yelp_sql = spark.sql("""SELECT  rest.name, rev.user_id, rev.text, rev.stars \
                             FROM reviews AS rev JOIN restaurants AS rest \
                             ON rev.business_id = rest.business_id""")

full_yelp_sql.show(10)

+--------------------+--------------------+--------------------+-----+
|                name|             user_id|                text|stars|
+--------------------+--------------------+--------------------+-----+
|             Levetto|TZQSUDDcA4ek5gBd6...|In the heart of C...|  4.0|
|Scaddabush Italia...|iFEM16O5BeKvf_AHs...|Was super excited...|  3.0|
|              Patria|Eg_VTAQwK3CxYTORN...|Excellent food, s...|  5.0|
|The Fish Store & ...|Kl6jNDAE7PG_VVQaf...|Wow, this baby's ...|  4.0|
|       Mangia & Bevi|IeojscL3rg3x3vtmR...|The pizza is very...|  4.0|
|  Queen St Warehouse|6WmMHMBM4FLGyK98-...|With many eaterie...|  1.0|
|Prohibition Gastr...|BwwqlPVsJk1dbUyNw...|Amazing place! My...|  5.0|
|Copacabana Brazil...|kmOvnwtroMBC8y9lO...|My wife is Brazil...|  1.0|
|            Our Spot|yiLiYYg6MM7Pmuo6j...|This place is ver...|  5.0|
|          Harbour 60|vxf427mKFMUxpfbHC...|This place gone d...|  2.0|
+--------------------+--------------------+--------------------+-----+
only s

The above data set is what will be used in the analysis.

The data set consists of the business names, users, reviews, and star ratings of the business under review.

In [21]:
# Export data set to project directory
full_yelp = full_yelp_sql.toPandas()
full_yelp.to_csv('Data/Yelp_Reviews_Data.csv')